In [2]:
!python -m pip install --upgrade pip
!pip install transformers
!pip install llama_index 
!pip install accelerate
!pip install sentence_transformers
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/5c/e9/ee572691a3fb05555bcde41826faad29ae4bc1fb07982e7f53d54a176879/scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/14/f2/10fa23f0a6b9b2439c01579ae4a9b1849d4822e972515c8f92584bfda5e9/scipy-1.11.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.9 MB/s eta 0:00:00

In [1]:
import logging
import sys, os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache/'
os.environ['HF_DATASETS_CACHE']='/workspace/cache/'

INFO:numexpr.utils:Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [2]:
from llama_index.prompts.prompts import SimpleInputPrompt

system_prompt = """<|SYSTEM|># A chat between a curious user and an artificial intelligence assistant.
The assistant gives helpful, detailed, and polite answers to the user's questions.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [3]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="ehartford/Wizard-Vicuna-13B-Uncensored",
    model_name="ehartford/Wizard-Vicuna-13B-Uncensored",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)
service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpy5mnaf8r
Created a temporary directory at /tmp/tmpy5mnaf8r
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpy5mnaf8r/_remote_module_non_scriptable.py
Writing /tmp/tmpy5mnaf8r/_remote_module_non_scriptable.py


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Supplied context_window 4096 is greater than the model's max input size {model_context_window}. Disable this warning by setting a lower context_window.


In [4]:
resp = llm.complete("Summarize the short story 'Gray Denim' by Harl Vincent.")
print(resp)

The story revolves around a young man named John who is obsessed with his appearance and material possessions. He meets a girl named Mary who is different from him in every way, but he is drawn to her. However, he ultimately rejects her and continues to pursue his superficial goals. The story highlights the dangers of being too focused on external appearances and material possessions at the expense of deeper values and relationships.


In [5]:
from pathlib import Path
from llama_index import download_loader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

# using the microsoft e5 embedding model

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
        model_name="intfloat/e5-large-v2",
        model_kwargs={"device": "cuda"},
    )
)

service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm, embed_model=embed_model)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-large-v2
Load pretrained SentenceTransformer: intfloat/e5-large-v2


In [7]:
EpubReader = download_loader("EpubReader")

loader = EpubReader()
documents = loader.load_data(file=Path("Super-Science-December-1930.epub"))


In [8]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./sfbook")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("Summarize the short story 'Gray Denim' by Harl Vincent.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
print(response)

The story is about a man named Karl who is a descendant of the Van Dorn family, a wealthy and powerful family in the United States. However, Karl has always felt out of place among his family and society, and he longs to escape and find his true calling. One day, he discovers a strange device that allows him to control gravity, and he uses it to fly away from his life and find his true home. He eventually lands on the coast of North America and meets a man named Rudolph Krassin, who gives him a suit of gray denim to wear. From then on, Karl lives a simple life as a worker in the lower levels of society, where he is content and happy.
The story takes place in a dystopian future where society is divided into two classes: the haves and the have-nots. The haves live in luxury while the have-nots struggle to survive in poverty. The protagonist, Karl, is a have-not who feels out of place among the wealthy elite. He discovers a strange device that allows him to control gravity, and he uses it

In [11]:
response = query_engine.query("Who are the Red Police in the short story 'Gray Denim' by Harl Vincent.")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Red Police in the short story 'Gray Denim' by Harl Vincent are a group of law enforcement officers who are responsible for maintaining order in the upper levels of society, where the wealthy and powerful live. They are described as being tall and muscular, and they wear red uniforms. They are also known for their use of advanced technology, such as pneumatic tubes and radiovision. In the story, the Red Police are portrayed as being ruthless and brutal, as they throw the protagonists into a cell and use force to interrogate them. It is implied that the Red Police are part of a larger organization that is planning to conquer the Earth.
